In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
spreadsheet = pd.ExcelFile('Dataset/dataset.xlsx')
df2 = spreadsheet.parse('Sheet1')
df2.head()

,State_Name,District_Name,Crop_Year,Season,Crop,Area(ha),Production(tonnes),Avg_Prep(mm),Avg_temp(C),Yield(tonnes/ha)
0,Maharashtra,AHMEDNAGAR,1997.0,Kharif,Arhar/Tur,17600.0,6300.0,136.30175,25.65,0.357955
1,Maharashtra,AHMEDNAGAR,1997.0,Kharif,Bajra,274100.0,152800.0,136.30175,25.65,0.557461
2,Maharashtra,AHMEDNAGAR,1997.0,Kharif,Gram,40800.0,18600.0,136.30175,25.65,0.455882
3,Maharashtra,AHMEDNAGAR,1997.0,Kharif,Jowar,900.0,1100.0,136.30175,25.65,1.222222
4,Maharashtra,AHMEDNAGAR,1997.0,Kharif,Maize,4400.0,4700.0,136.30175,25.65,1.068182


In [3]:
crops_combo=[]
crop_season=[]
for row in df2.itertuples():
    #use row[5] strip
    crop_season.append(str(row[5]).strip())
    #use row[4] strip
    crop_season.append(str(row[4]).strip())
    crops_combo.append(crop_season)
    crop_season=[]
a = set(tuple(i) for i in crops_combo)
print('length of set :',len(a))
for l in sorted(a):
    print(l[0]," ",l[1])

length of set : 41
Arhar/Tur   Kharif
Bajra   Kharif
Castor seed   Kharif
Cotton(lint)   Kharif
Cotton(lint)   Whole Year
Gram   Kharif
Gram   Rabi
Groundnut   Kharif
Groundnut   Summer
Jowar   Kharif
Jowar   Rabi
Linseed   Rabi
Maize   Kharif
Maize   Rabi
Maize   Summer
Moong(Green Gram)   Kharif
Niger seed   Kharif
Other  Rabi pulses   Rabi
Other Cereals & Millets   Kharif
Other Cereals & Millets   Rabi
Other Kharif pulses   Kharif
Pulses total   Kharif
Ragi   Kharif
Rapeseed &Mustard   Rabi
Rice   Kharif
Rice   Summer
Safflower   Rabi
Sesamum   Kharif
Sesamum   Rabi
Small millets   Kharif
Small millets   Rabi
Soyabean   Kharif
Sugarcane   Kharif
Sugarcane   Whole Year
Sunflower   Kharif
Sunflower   Rabi
Sunflower   Summer
Total foodgrain   Kharif
Urad   Kharif
Wheat   Rabi
nan   nan


In [5]:
ar = []
ar2d = []

x_list=[]
z_list=[]
i=0
y_list=[]

model_list = []
for l in sorted(a):
    for row in df2.itertuples():
        if str(row[5]).strip()==l[0] and str(row[4]).strip()==l[1]:
            ar.append(row[8])
            x_list.append(row[8])
            ar.append(row[9])
            z_list.append(row[9])
            y_list.append(row[10])
            ar2d.append(ar)
            ar = []
    if len(ar2d) < 10:
        continue
    
    nar2d = np.array(ar2d)
    nar2d = np.nan_to_num(nar2d)
    ny = np.array(y_list)
    ny = np.nan_to_num(ny)
    
    X_train = nar2d[:-4]
    X_test = nar2d[-4:]

    y_train = ny[:-4]
    y_test = ny[-4:]

    # Fit line using all data
    lr = linear_model.LinearRegression()
    lr.fit(X_train, y_train)
    
    line_y = lr.predict(X_test)

    # Robustly fit linear model with RANSAC algorithm
    ransac = linear_model.RANSACRegressor()
    ransac.fit(X_train, y_train)
    inlier_mask = ransac.inlier_mask_
    outlier_mask = np.logical_not(inlier_mask)

    line_y_ransac = ransac.predict(X_test)
    
    model_list.append([ransac,l[0],l[1]])
    #print("ransac prediction")
    #print(line_y_ransac)
    # Compare estimated coefficients
    #print("Estimated coefficients (linear regression, RANSAC):")
    #print(lr.coef_, ransac.estimator_.coef_)

    # The mean squared error
    print("Mean squared error of linear regression: %.2f"
          % mean_squared_error(y_test, line_y))
    print("Mean squared error of ramsac regression: %.2f"
          % mean_squared_error(y_test, line_y_ransac))
    # Explained variance score: 1 is perfect prediction
    print('Variance score of linear regr: %.2f' % r2_score(y_test, line_y))
    print('Variance score of ransac: %.2f' % r2_score(y_test, line_y_ransac))

Mean squared error of linear regression: 0.23
Mean squared error of ramsac regression: 0.14
Variance score of linear regr: -23.78
Variance score of ransac: -14.70
Mean squared error of linear regression: 0.04
Mean squared error of ramsac regression: 0.01
Variance score of linear regr: -2.38
Variance score of ransac: -0.24
Mean squared error of linear regression: 0.14
Mean squared error of ramsac regression: 0.06
Variance score of linear regr: -18.58
Variance score of ransac: -7.75
Mean squared error of linear regression: 0.03
Mean squared error of ramsac regression: 0.03
Variance score of linear regr: 0.08
Variance score of ransac: -0.01
Mean squared error of linear regression: 21406.15
Mean squared error of ramsac regression: 21531.45
Variance score of linear regr: -2.13
Variance score of ransac: -2.15
Mean squared error of linear regression: 19.44
Mean squared error of ramsac regression: 0.07
Variance score of linear regr: -2055.19
Variance score of ransac: -5.99
Mean squared error o

In [6]:
def yieldPredict(test_input):
    y_predicted = []
    for m in model_list:
        ys = m[0].predict(test_input)
        y_predicted.append([ys,m[1],m[2]])
        #print(m[1], m[2])
        #print(ys)
    y_predicted.sort(reverse=True)
    for i in y_predicted:
        print(i[0], " ", i[1], " ", i[2])
        
yieldPredict([[1,25]])

[0.75734036]   Moong(Green Gram)   Kharif
[0.74128922]   Maize   Summer
[0.70575151]   Groundnut   Summer
[0.68025145]   Other Cereals & Millets   Rabi
[0.61690073]   Gram   Rabi
[0.616347]   Maize   Kharif
[0.61057687]   Rice   Summer
[0.60990234]   Jowar   Kharif
[0.59240424]   Groundnut   Kharif
[0.58413265]   Jowar   Rabi
[0.56602903]   nan   nan
[0.54871496]   Other Cereals & Millets   Kharif
[0.53474888]   Other Kharif pulses   Kharif
[0.52849607]   Total foodgrain   Kharif
[0.52695661]   Safflower   Rabi
[0.52136269]   Soyabean   Kharif
[0.50593219]   Sesamum   Kharif
[0.50547162]   Sugarcane   Kharif
[0.50146689]   Urad   Kharif
[0.49396945]   Wheat   Rabi
[0.45742756]   Sugarcane   Whole Year
[0.44733909]   Linseed   Rabi
[0.41853541]   Rice   Kharif
[0.41350454]   Other  Rabi pulses   Rabi
[0.41105129]   Maize   Rabi
[0.40195214]   Pulses total   Kharif
[0.38056004]   Small millets   Kharif
[0.37314911]   Cotton(lint)   Kharif
[0.36807365]   Ragi   Kharif
[0.36719988]   Niger